# Google Colab R

- Google is now supporting a Colab notebook with the R kernel
- We can run R codes directly on Colab
- R Colab Notebook Link: [R Google Colab](https://colab.fan/r)
    - Colab with R kernel
    - With base-R installed
    - Run R codes immediately
    - No need to setup
    - Save a copy in your Drive


## Demonstration

This notebook uses the R kernel.


In [1]:
sessionInfo()

R version 4.5.2 (2025-10-31)
Platform: x86_64-pc-linux-gnu
Running under: Ubuntu 22.04.5 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas-pthread/libblas.so.3 
LAPACK: /usr/lib/x86_64-linux-gnu/openblas-pthread/libopenblasp-r0.3.20.so;  LAPACK version 3.10.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] digest_0.6.39     IRdisplay_1.1     base64enc_0.1-3   fastmap_1.2.0    
 [5] glue_1.8.0        htmltools_0.5.9   repr_1.1.7        lifecycle_1.0.5  
 [9] cl

In [2]:
# TOTES LES COMBINACIONS FINS UN MOMENT CONCRET

# https://cran.r-project.org/web/packages/magrittr/index.html
`while`(!"magrittr" %in% installed.packages()[, "Package"],
        install.packages("magrittr", dependencies = TRUE))

library(magrittr) # adfñdsjkfñldjañkfjdfñlkjfadñkdjfafñlkdsjfñ

# https://cran.r-project.org/web/packages/fasttime/index.html
`while`(!"fasttime" %in% installed.packages()[, "Package"],
        install.packages("fasttime", dependencies = TRUE))

# https://cran.r-project.org/web/packages/readr/index.html
# https://cran.r-project.org/web/packages/dplyr/index.html
# https://cran.r-project.org/web/packages/lubridate/index.html
c("readr", "dplyr", "lubridate") %>%
`for`(package, ., library(package, character.only = TRUE))

# reticulate::py$EURUSD_M1_2020 %>% # https://uc-r.github.io/pipe
read_delim("DAT_ASCII_EURUSD_M1_2020.csv", delim = ";",
           escape_double = FALSE, trim_ws = TRUE, show_col_types = FALSE,
           col_names = c("Data", "Apertura", "Màxim", "Mínim", "Tancament"),
           col_select = c(Data = 1, Apertura = 2, Màxim = 3, Mínim = 4, Tancament = 5)) |> arrange(desc(row_number())) |>
  mutate(Data = format(Data, "%Y-%m-%d %H:%M")) |> # (no fa falta si 10 i 16 estan actividades)
  # rename(`Màxim` = Maxim, `Mínim` = Minim) %>%      # hours(1) e también hours(2)
  # mutate(Data = fasttime::fastPOSIXct(Data, tz = "UCT") + hours(7), DiaSet = wday(Data, week_start = 1), Hora = hour(Data), Minut = minute(Data), Data = as.Date(Data),
  #       `Max-Aper` = `Màxim` - Apertura, `Aper-Min` = Apertura - `Mínim`, `Tanc-Aper` = Tancament - Apertura, `lead(Tanc-Aper)` = lead(`Tanc-Aper`),
  #       Sentit = case_when(`Tanc-Aper` == 0 ~ "NEUTRAL", `Tanc-Aper` > 0 ~ "UP", `Tanc-Aper` < 0 ~ "DOWN")) |>
  mutate(Data = fasttime::fastPOSIXct(Data, tz = "UCT"), DiaSet = wday(Data, week_start = 1), Hora = hour(Data), Minut = minute(Data), Data = as.Date(Data),
        `Max-Aper` = `Màxim` - Apertura, `Aper-Min` = Apertura - `Mínim`, `Tanc-Aper` = Tancament - Apertura, `lead(Tanc-Aper)` = lead(`Tanc-Aper`),
        Sentit = case_when(`Tanc-Aper` == 0 ~ "NEUTRAL", `Tanc-Aper` > 0 ~ "UP", `Tanc-Aper` < 0 ~ "DOWN")) |>
  relocate(DiaSet, Hora, Minut, .before = Apertura) %T>%
  {
    group_by(.data = ., Data) |>
      summarise(Apertura = tail(Apertura, 1), `Màxim` = max(`Màxim`), `Mínim` = min(`Mínim`), Tancament = head(Tancament, 1), .groups = "drop") |>
      arrange(desc(Data)) |>
      mutate(`Max-Aper` = `Màxim` - Apertura, `Aper-Min` = Apertura - `Mínim`, `Tanc-Aper` = Tancament - Apertura,
             `lead(Tanc-Aper)` = lead(`Tanc-Aper`), Sentit = case_when(`Tanc-Aper` == 0 ~ "NEUTRAL", `Tanc-Aper` > 0 ~ "UP", `Tanc-Aper` < 0 ~ "DOWN")) ->> dayinfo
  } |>
  group_by(Data, DiaSet, Hora) %T>%
  {
    summarise(.data = ., Apertura = last(Apertura), `Màxim` = max(`Màxim`), `Mínim` = min(`Mínim`), Tancament = first(Tancament), .groups = "drop") |>
      arrange(desc(Data), desc(Hora)) |>
      mutate(`Max-Aper` = `Màxim` - Apertura, `Aper-Min` = Apertura - `Mínim`, `Tanc-Aper` = Tancament - Apertura,
             `lead(Tanc-Aper)` = lead(`Tanc-Aper`), Sentit = case_when(`Tanc-Aper` == 0 ~ "NEUTRAL", `Tanc-Aper` > 0 ~ "UP", `Tanc-Aper` < 0 ~ "DOWN")) ->> hourinfo
  } |> # pick(everything()) = cur_data()
  summarise(AltresVar = list(pick(everything())), NMinuts = n(), MinOblidats = paste(c(59:0)[!(c(59:0) %in% Minut)], collapse = "-"), .groups = "drop_last") |>
  arrange(desc(Data), desc(Hora)) |>
  left_join(hourinfo, by = c("Data", "Hora"), suffix = c("", "")) |>
  summarise(AltresVar = list(pick(everything())), NHores = n(), HorOblidades = paste(c(23:0)[!(c(23:0) %in% Hora)], collapse = "-"), NMinuts = sum(NMinuts), .groups = "drop_last") |>
  arrange(desc(Data)) |>
  left_join(dayinfo, by = "Data") -> simplified_EURUSD_M1_2020

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




https://scales.arabpsychology.com/stats/how-to-add-days-to-date-in-r-with-examples/ \
https://stackoverflow.com/questions/18537049/fastposixct-equivalent-for-converting-non-utc-to-utc \
https://www.farmersalmanac.com/utc-time-and-gmt-time#:~:text=Although%20both%20GMT%20and%20UTC,time%20and%20time%20zones%20worldwide. \
https://www.elperiodico.com/es/sociedad/20240330/cambio-de-hora-2024-espana-horario-verano-dv-7368071

In [ ]:
# simplified_EURUSD_M1_2020[1:2, ]

In [3]:
simplified_EURUSD_M1_2020 %>% pull(AltresVar) %>% first() %>% pull(AltresVar) %>% first()

Minut,Apertura,Màxim,Mínim,Tancament,Max-Aper,Aper-Min,Tanc-Aper,lead(Tanc-Aper),Sentit
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
58,1.22163,1.22168,1.22143,1.22153,0.00005,0.00020,-0.00010,-0.00002,DOWN
57,1.22166,1.22167,1.22153,1.22164,0.00001,0.00013,-0.00002,-0.00011,DOWN
56,1.22176,1.22179,1.22160,1.22165,0.00003,0.00016,-0.00011,-0.00001,DOWN
55,1.22175,1.22180,1.22170,1.22174,0.00005,0.00005,-0.00001,0.00016,DOWN
54,1.22159,1.22182,1.22153,1.22175,0.00023,0.00006,0.00016,-0.00011,UP
53,1.22170,1.22170,1.22157,1.22159,0.00000,0.00013,-0.00011,0.00004,DOWN
52,1.22166,1.22173,1.22166,1.22170,0.00007,0.00000,0.00004,0.00000,UP
51,1.22166,1.22175,1.22166,1.22166,0.00009,0.00000,0.00000,-0.00005,NEUTRAL
50,1.22171,1.22178,1.22161,1.22166,0.00007,0.00010,-0.00005,-0.00002,DOWN


In [ ]:
# @title
simplified_EURUSD_M1_2020 |>
  select(-AltresVar) |>
# dayinfo %>%
  # =(QUARTILE.INC((D3,F4,F5),0)*G4+QUARTILE.INC((D3,(C4:F5)),4)*G5)/2
  within({
    (
      (pmin(Mínim, lead(Mínim)) * `lead(Tanc-Aper)`) +
      (pmax(Màxim, lead(Màxim)) * `Tanc-Aper`)
    ) / 2 -> Algorisme
  }) |>
  mutate(
    Previsió = Algorisme + Apertura,
    Retenció = ifelse(Previsió > Apertura, Apertura + 0.0016, Apertura - 0.0016),
    Revisió =
      ifelse(
        Retenció >= `Mínim` & Retenció <= `Màxim`,
        ifelse(Retenció > Apertura, Retenció - Apertura, Apertura - Retenció),
        ifelse(Retenció > Apertura, `Tanc-Aper`, Apertura - Tancament)
      ),
	Etiquetació = ifelse(Retenció >= Apertura, TRUE, FALSE)
  ) |>
  filter(`Max-Aper` >= 0.0016 | `Aper-Min` >= 0.0016) |>
  filter((Etiquetació & `Aper-Min` >= 0.0016) | (!Etiquetació & `Max-Aper` >= 0.0016)) |>
  # filter(`Aper-Min` >= 0.0016 & `Max-Aper` >= 0.0016) |> # (només serveix per contar, però no es pot considerar)
  within({
	Data ->> Dates
  })

In [ ]:
# @title
hourinfo |>
  filter(Data %in% Dates)

Referència: https://cran.r-project.org/web/packages/colorDF/vignettes/colorDF.html#7_Setting_up_colorDF_as_the_default_data_frame_print_method

In [13]:
nrow(dayinfo)
# str(simplified_EURUSD_M1_2020) # RETORNA L'ESTRUCTURA

[1] 314

In [8]:
# buscar otro visualizador no tant pesado.

# `while`(!"reactable" %in% installed.packages()[, "Package"], install.packages("reactable", dependencies = TRUE))

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘proxy’, ‘e1071’, ‘wk’, ‘sp’, ‘terra’, ‘classInt’, ‘s2’, ‘units’, ‘rex’, ‘leaflet.providers’, ‘png’, ‘raster’, ‘sf’, ‘reactR’, ‘covr’, ‘leaflet’, ‘sparkline’, ‘tippy’, ‘V8’


Warning message in install.packages("reactable", dependencies = TRUE):
“installation of package ‘s2’ had non-zero exit status”
Warning message in install.packages("reactable", dependencies = TRUE):
“installation of package ‘raster’ had non-zero exit status”
Warning message in install.packages("reactable", dependencies = TRUE):
“installation of package ‘covr’ had non-zero exit status”
Warning message in install.packages("reactable", dependencies = TRUE):
“installation of package ‘classInt’ had non-zero exit status”
Warning message in install.packages("reactable", dependencies = TRUE):
“installation of package ‘sf’ had non-zero exit status”
Warning message in install.packages("reactable", dependencies 